In [1]:
# Import dependencies
import numpy as np
from cellpose_omni import models, core
from cellpose_omni import plot
import omnipose
import zarr

# This checks to see if you have set up your GPU properly.
# CPU performance is a lot slower, but not a problem if you 
# are only processing a few images.
use_GPU = core.use_gpu()
print('>>> GPU activated? {}'.format(use_GPU))

# for plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['figure.dpi'] = 300
plt.style.use('dark_background')
%matplotlib inline

2024-04-05 17:36:18,987 [INFO] No OpenGL_accelerate module loaded: No module named 'OpenGL_accelerate'
2024-04-05 17:36:18,997 [INFO] Failed to load library ( 'libOpenGL.so.0' ): libOpenGL.so.0: cannot open shared object file: No such file or directory
2024-04-05 17:36:19,232 [INFO] TORCH GPU version not installed/working.
>>> GPU activated? False


In [2]:
# model_name = "kieran_td/models/cellpose_residual_on_style_on_concatenation_off_omni_abstract_nclasses_2_nchan_1_dim_2_kieran_td_new_2024_03_05_18_01_51.434305_epoch_3998"
model_name = "trench_masks/models/cellpose_residual_on_style_on_concatenation_off_omni_abstract_nclasses_2_nchan_1_dim_2_trench_masks_2024_03_29_01_09_11.399848_epoch_3999"
print(model_name)

trench_masks/models/cellpose_residual_on_style_on_concatenation_off_omni_abstract_nclasses_2_nchan_1_dim_2_trench_masks_2024_03_29_01_09_11.399848_epoch_3999


In [3]:
use_gpu = use_GPU

In [4]:
model = models.CellposeModel(gpu=use_gpu, pretrained_model=model_name, nchan=1, nclasses=2,dim=2)
epoch = model_name.split("_")[-1]

2024-04-05 17:36:19,349 [INFO] >>>> using CPU


In [5]:
chans = None

# define parameters
mask_threshold = -1
verbose = 0 # turn on if you want to see more output
use_gpu = use_GPU #defined above
transparency = True # transparency in flow output
rescale=None # give this a number if you need to upscale or downscale your images
omni = True # we can turn off Omnipose mask reconstruction, not advised
flow_threshold = 0 # default is .4, but only needed if there are spurious masks to clean up; slows down output
resample = True #whether or not to run dynamics on rescaled grid or original grid

In [6]:
from numcodecs import Blosc
compressor = Blosc(cname='zstd', clevel=9, shuffle=Blosc.BITSHUFFLE)

In [7]:
## Load Trenches

In [8]:
trenches_zarr = zarr.open("/mnt/c/Users/kiera/one_frame.zarr", mode='r')

In [9]:
trenches_zarr.shape

(68, 14, 3, 1, 600, 100)

In [10]:
# fov_range = list(range(0, 31)) # FOV range to process
fov_range = list(range(40, 68)) # FOV range to process
num_fovs = 68
num_trenches = 14
num_frames = 1
tile_length = 1

In [11]:
concat_zarr = zarr.open("concat.zarr", 
                       mode="w", 
                       shape=(num_fovs*num_trenches*num_frames/tile_length, 600, 100*tile_length),
                       chunks=(1, 600, 100*tile_length),
                       dtype='uint16', 
                       compressor = compressor)

In [12]:
masks_zarr = zarr.open("masks.zarr", 
                       mode="w", 
                       shape=(num_fovs*num_trenches*num_frames/tile_length, 600, 100*tile_length),
                       chunks=(1, 600, 100*tile_length),
                       dtype='uint16', 
                       compressor = compressor)

In [ ]:
# concat_zarr = zarr.open("concat_280324.zarr", 
#                        mode="r+")

In [ ]:
# masks_zarr = zarr.open("masks_280324.zarr", 
#                        mode="r+")

In [14]:
fail_list = []

for fov_index in fov_range:
    for trench_index in range(num_trenches):
        for start_frame in range(0, num_frames, tile_length):
            end_frame = min(start_frame + tile_length, num_frames)
            zarr_index = (fov_index * num_trenches * (num_frames // tile_length) + trench_index * (num_frames // tile_length) + start_frame // tile_length)
                
            try:
                trench_timeseries = trenches_zarr[fov_index,trench_index,0,start_frame:end_frame,:,:]
                trench_timeseries = [x for x in trench_timeseries]
                concat_trench_timeseries = np.concatenate(trench_timeseries, axis = 1)
                imgs = [concat_trench_timeseries]
                nimg = len(imgs)
                n = range(nimg)

                masks, flows, styles = model.eval([imgs[i] for i in n],
                                      channels=chans,
                                      rescale=rescale,
                                      mask_threshold=mask_threshold,
                                      transparency=transparency,
                                      flow_threshold=flow_threshold,
                                      omni=omni,
                                      resample=resample,
                                      verbose=verbose,
                                     )

                concat_zarr[zarr_index, :, :] = concat_trench_timeseries
                masks_zarr[zarr_index, :, :] = masks[0]
                print(f"Tile {zarr_index} Succeeded")
            
            except:
                fail_list.append(zarr_index)
                print(f"Tile {zarr_index} Failed")
                
print("Failed:")
print(fail_list)

Tile 560 Succeeded
Tile 561 Succeeded
Tile 562 Succeeded
Tile 563 Succeeded
Tile 564 Succeeded
2024-04-05 17:36:40,937 [INFO] No cell pixels found.
Tile 565 Succeeded
2024-04-05 17:36:42,333 [INFO] No cell pixels found.
Tile 566 Succeeded
2024-04-05 17:36:44,015 [INFO] No cell pixels found.
Tile 567 Succeeded
Tile 568 Succeeded
Tile 569 Succeeded
2024-04-05 17:36:48,164 [INFO] No cell pixels found.
Tile 570 Succeeded
Tile 571 Succeeded
Tile 572 Succeeded
Tile 573 Succeeded
Tile 574 Succeeded
Tile 575 Succeeded
Tile 576 Succeeded
2024-04-05 17:36:59,113 [INFO] No cell pixels found.
Tile 577 Succeeded
Tile 578 Succeeded
Tile 579 Succeeded
Tile 580 Succeeded
Tile 581 Succeeded
Tile 582 Succeeded
Tile 583 Succeeded
Tile 584 Succeeded
Tile 585 Succeeded
Tile 586 Succeeded
Tile 587 Succeeded
Tile 588 Succeeded
Tile 589 Succeeded
Tile 590 Succeeded
Tile 591 Succeeded
Tile 592 Succeeded
Tile 593 Succeeded
Tile 594 Succeeded
Tile 595 Succeeded
Tile 596 Succeeded
Tile 597 Succeeded
Tile 598 Succ

In [5]:
# concat_zarr = zarr.open("concat_310324.zarr", 
#                        mode="r")
# masks_zarr = zarr.open("masks_310324.zarr", 
#                        mode="r")

In [7]:
files = len(masks_zarr)

In [32]:
files

8425

In [33]:
concat_zarr_split = zarr.open("concat_split.zarr", 
                       mode="w", 
                       shape=(files*tile_length, 600, 100),
                       chunks=(1, 600, 100),
                       dtype='uint16', 
                       compressor = compressor)

In [35]:
concat_split_list = []
for concat_array in concat_zarr:
    concat_succeeded_split = np.split(concat_array, tile_length, axis=-1)
    concat_split_list.append(concat_succeeded_split)

concat_combined = np.concatenate(concat_split_list, axis=0)

concat_zarr_split[:] = concat_combined

In [8]:
masks_zarr_split = zarr.open("masks_split.zarr", 
                       mode="w", 
                       shape=(files*tile_length, 600, 100),
                       chunks=(1, 600, 100),
                       dtype='uint16', 
                       compressor = compressor)

In [9]:

masks_split_list = []
for mask_array in masks_zarr:
    masks_succeeded_split = np.split(mask_array, tile_length, axis=-1)
    masks_split_list.append(masks_succeeded_split)

masks_combined = np.concatenate(masks_split_list, axis=0)

masks_zarr_split[:] = masks_combined

In [10]:
empty_zarr = zarr.open("empty.zarr", 
                       mode="w", 
                       shape=(files*tile_length, 600, 100),
                       chunks=(1, 600, 100),
                       dtype='uint8', 
                       compressor = compressor)